In [1]:
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import sys
sys.path.append('/home/li3975/cs587/Pre-Processing-Fair-Representation/')

from model.utils.metrics import *
from model.utils.dataloader import dataloader
from model.LFR import LFR
import matplotlib.pyplot as plt
import numpy as np
import random

# Set random seeds for reproducibility
random.seed(42)  # Fixes the seed for Python's random module to ensure consistent results
np.random.seed(42)  # Fixes the seed for NumPy's random module for consistency


sensitive_features = {'credit': 'V5', 
                      'student':  'Family_Income',
                      'kiva': 'Borrower_genders', 
}



dataSet = "student"
sensitive_feature = sensitive_features[dataSet]
# Load and preprocess the dataset using a custom dataloader
# Dataloader returns the dataset, target labels, numerical, and categorical features

data = dataloader(dataSet, sensitive_feature) 
dataset, target, numvars, categorical = data  # Unpack the returned values

# print(dataset)
# print(target)


# Split the dataset into training and testing sets with stratified sampling to maintain target distribution
x_train, x_test, y_train, y_test = train_test_split(dataset,
                                                    target,
                                                    test_size=0.1,
                                                    random_state=42)



# Remove the sensitive feature ('gender') from the classification target
classification = target.columns.to_list()  # Get a list of target columns
classification.remove(sensitive_feature)  # Remove the sensitive feature
classification = classification[0]  # Select the primary classification target feature

# Create a pipeline to standardize numerical features
numeric_transformer = Pipeline(
    steps=[('scaler', StandardScaler())])  # Standardizes numerical data by removing the mean and scaling to unit variance

# Create a pipeline to one-hot encode categorical features
categorical_transformer = Pipeline(
    steps=[('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))])  # Transforms categorical variables into binary vectors

# Combine numerical and categorical transformations
transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numvars),  # Apply numeric transformation to numeric columns
        ('cat', categorical_transformer, categorical)])  # Apply categorical transformation to categorical columns

# Build a pipeline that preprocesses both numerical and categorical data
pipeline = Pipeline(steps=[('preprocessor', transformations)])  
dict_all_result_in_grid = {}  # Initialize an empty dictionary to store grid search results

# Apply the transformation pipeline to the training data
x_train = pipeline.fit_transform(x_train)

# Set parameters for the LFR fairness transformation model
parameters = {'k': 10, 'Ax': 0.001, 'Ay': 0.1, 'Az': 10.0, 'max_iter': 150000, 'max_fun': 150000}

# Instantiate the LFR (Learning Fair Representations) model
lfr = LFR(sensitive_feature=sensitive_feature, privileged_class=1, unprivileged_class=0, seed=42,
            output_feature=classification, parameter=parameters)

# Train the LFR model on the transformed training data
lfr.fit(X=x_train, y=y_train)

# Transform the training data into fair representations using LFR
Z_train, y_trainLFR = lfr.transform(X=x_train, y=y_train)

# Initialize arrays to store results for accuracy and fairness metrics
bal_acc_arr_transf = []
deo_arr_transf = []
dao_arr_transf = []
FairDEO = []
FairDAO = []

# Define a range of thresholds to evaluate the trade-off between accuracy and fairness
thresholds = np.linspace(0.01, 0.99, 100)

# Initialize and train an SVM classifier with a linear kernel on the transformed training data
svc = SVC(kernel='linear')
svc.fit(Z_train, y_train[classification])

# Apply the transformation pipeline to the test data
x_test = pipeline.transform(x_test)

# Transform the test data using the LFR model
Z_test, y_testLFR = lfr.transform(X=x_test, y=y_test)

# Predict the classification labels for the test set using the trained SVM
y_pred = svc.predict(Z_test)

# Calculate accuracy of the SVM predictions on the test set
ACC = accuracy_score(y_pred, y_test[classification])

# Loop through each threshold and evaluate the accuracy and fairness metrics
for thresh in thresholds:
    Y_pred = y_trainLFR.copy()  # Copy the training labels after LFR transformation
    Y_pred[classification] = np.array(Y_pred[classification] > thresh).astype(np.float64)  # Apply threshold to predictions
    ACC = accuracy_score(y_train[classification], Y_pred[classification])  # Calculate accuracy
    DEO = DifferenceEqualOpportunity(Y_pred[classification], y_train, sensitive_feature, classification, 1, 0, [0, 1])  # Equal opportunity difference
    DAO = DifferenceAverageOdds(Y_pred[classification], y_train, sensitive_feature, classification, 1, 0, [0, 1])  # Average odds difference
    bal_acc_arr_transf.append(ACC)
    deo_arr_transf.append(DEO)
    dao_arr_transf.append(DAO)
    FairDEO.append(ACC * (1 - DEO))  # Weighted fairness with DEO
    FairDAO.append(ACC * (1 - DAO))  # Weighted fairness with DAO



/home/li3975/cs587/Pre-Processing-Fair-Representation/model/utils/dataloader.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target.replace(['High', 'Medium', 'Low'], [2, 1, 0], inplace=True)


>>>>>>>>fit_called<<<<<<<<<<
250 12501.637984834588
500 12501.596016632542
750 -13483.221421406715
1000 -13483.223002110688
1250 -108864.12006767662
1500 -108864.07806226703
1750 -108864.08069454822
2000 -130909.75482814513
2250 -130909.749419526
2500 -139701.46001165887
2750 -139701.43081849933
3000 -139701.45437986162
3250 -154805.15667087122
3500 -154805.1517506894
3750 -157141.01220768603
4000 -157140.9972835059
4250 -157141.0059213479
4500 -158913.0147377762
4750 -158912.98372212632
5000 -160062.00011805614
5250 -160061.99172011373
5500 -160061.96836385503
5750 -161345.66702805844
6000 -161345.67232056145
6250 -163247.90210560986
6500 -163247.90557934612
6750 -163247.90261661628
7000 -166286.95386653516
7250 -166286.9637732659
7500 -172002.11146548772
7750 -172002.10549788468
8000 -172002.13443206358
8250 -209700.40649713384
8500 -209700.40647893574
8750 -209701.15400391776
9000 -209701.1540038659
9250 -209701.15400380906
9500 -209704.1368837469
9750 -209704.1374679228
10000 -2097

ValueError: At least one label specified must be in y_true

In [ ]:
# Plot the trade-off between accuracy and fairness metrics for the training set
plt.title("tradeOff Accuracy-Fairness for different thresholds (TRAIN)")
plt.plot(thresholds, bal_acc_arr_transf, marker='.')
plt.plot(thresholds, deo_arr_transf, marker='.')
plt.plot(thresholds, dao_arr_transf, marker='.')
plt.plot(thresholds, FairDEO, marker='.')
plt.plot(thresholds, FairDAO, marker='.')
plt.ylim(0, 1)
plt.legend(["ACC", "DEO", "DAO", "F_DEO", "F_DAO"])
plt.xlabel("threshold")
plt.show()



In [ ]:
# Clear arrays to store test results
bal_acc_arr_transf = []
deo_arr_transf = []
dao_arr_transf = []
FairDEO = []
FairDAO = []

# Evaluate accuracy and fairness metrics on the test data
for thresh in thresholds:
    Y_pred = y_testLFR.copy()
    Y_pred[classification] = np.array(Y_pred[classification] > thresh).astype(np.float64)
    ACC = accuracy_score(y_test[classification], Y_pred[classification])
    DEO = DifferenceEqualOpportunity(Y_pred[classification], y_test, sensitive_feature, classification, 1, 0, [0, 1])
    DAO = DifferenceAverageOdds(Y_pred[classification], y_test, sensitive_feature, classification, 1, 0, [0, 1])
    bal_acc_arr_transf.append(ACC)
    deo_arr_transf.append(DEO)
    dao_arr_transf.append(DAO)
    FairDEO.append(ACC * (1 - DEO))
    FairDAO.append(ACC * (1 - DAO))

# Plot the trade-off between accuracy and fairness metrics for the test set
plt.title("tradeOff Accuracy-Fairness for different thresholds (TEST)")
plt.plot(thresholds, bal_acc_arr_transf, marker='.')
plt.plot(thresholds, deo_arr_transf, marker='.')
plt.plot(thresholds, dao_arr_transf, marker='.')
plt.plot(thresholds, FairDEO, marker='.')
plt.plot(thresholds, FairDAO, marker='.')
plt.ylim(0, 1)
plt.legend(["ACC", "DEO", "DAO", "F_DEO", "F_DAO"])
plt.xlabel("threshold")
plt.show()

# Print accuracy and fairness metrics based on Zemel's method
print("Next the metrics used by zemel, et al.")
print("Accuracy: {}".format(accuracy(y_test[classification], y_testLFR[classification])))
print("Discrimination: {}".format(discrimination(
    y_test, y_testLFR[classification], sensitive_feature, 1, 0)))
print("Consistency: {}".format(consistency(x_test, y_testLFR[classification], k=5)))
